# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
path = '../output_data/cities.csv'

cities = pd.read_csv(path)

cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Henties Bay,-22.1160,14.2845,68.0,88,91,5.10,NaN,1611716964
1,Bethel,41.3712,-73.4140,28.0,93,90,4.61,US,1611716757
2,Saint-François,46.4154,3.9054,33.8,80,0,5.75,FR,1611716964
3,Ushuaia,-54.8000,-68.3000,39.2,81,75,23.02,AR,1611716749
4,Mar del Plata,-38.0023,-57.5575,71.6,83,75,4.61,AR,1611716964


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
locations = cities[['Lat', 'Lng']]
weights = cities['Humidity'].astype(int)

# Layout
figure_layout = {
     'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
ideal_vacay = cities.loc[((cities['Max Temp'] <=75) & (cities ['Max Temp'] >=55)) & (cities['Humidity'] <=55) & 
                        (cities['Wind Speed'] <=20)]
ideal_vacay.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Busselton,-33.6500,115.3333,70.00,48,1,8.01,AU,1611716967
45,New Norfolk,-42.7826,147.0587,61.00,44,75,14.97,AU,1611716970
70,Hobart,-42.8794,147.3294,59.00,44,75,14.97,AU,1611716872
124,Ferkessédougou,9.6000,-5.2000,69.24,23,0,2.44,CI,1611716980
137,Panzhihua,26.5851,101.7128,55.99,41,3,1.01,CN,1611716981


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
ideal = ideal_vacay.copy()
ideal['Hotel Name'] = ""
ideal.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,Busselton,-33.6500,115.3333,70.00,48,1,8.01,AU,1611716967,
45,New Norfolk,-42.7826,147.0587,61.00,44,75,14.97,AU,1611716970,
70,Hobart,-42.8794,147.3294,59.00,44,75,14.97,AU,1611716872,
124,Ferkessédougou,9.6000,-5.2000,69.24,23,0,2.44,CI,1611716980,
137,Panzhihua,26.5851,101.7128,55.99,41,3,1.01,CN,1611716981,


In [ ]:
# how to associate found hotels with city names and append to data frame for next step
# some cities did not have hotels...

In [26]:
hotels = []

# target_coordinates = list(zip(ideal.Lat, ideal.Lng))
target_radius = 5000
target_type = 'lodging'

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i,row in ideal.iterrows():
    params = {
    'location': (f"{row['Lat']},{row['Lng']}"),
    'radius': target_radius,
    'types': target_type,
    'key': g_key
}
    response = requests.get(base_url, params=params).json()
#     print(response['results'][0]['name'])
    
    try:
        hotels.append(response['results'][0]['name'])
    except:
        print('not found')
        
print(hotels)


not found
not found
not found
not found
not found
['Hotel The Royal Honours', 'OYO 9856 Hotel MP Inn', 'Hotel Boutique Las Azucenas', 'Circuit House', 'Dibbo areaمنطقة دبو', 'A.B.U Hotels', 'Best Western Premier Bryan College Station', 'Villa Madrid', 'Gayatri Auto advisor', 'Mövenpick Hotel Qassim', 'Sinya on Lone Man Creek', 'Hotel Tesoro Los Cabos', 'Hotel Arena Gruesa', 'RAMAAS HOTEL', 'Nuba Nile', 'Expeditions Ténére voyages', 'Hotel Narayan Avezika Comfort', 'محمد هادي ال فطيح', 'Beau Site', 'Lords Inn Porbandar', 'Case de Passage De la Prefecture', 'صاحبة الفخامة']


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
locations = hotel_df[['Lat', 'Lng']]
weights = hotel_df['Humidity'].astype(int)

# Layout
figure_layout = {
     'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig